# Lab 03 Part B Notebook

Use this notebook to familiarize yourself with the flex flow syntax. Note the following references. 

- CLI reference: https://microsoft.github.io/promptflow/reference/pf-command-reference.html
- SDK on getting started with flex flow: https://microsoft.github.io/promptflow/tutorials/flex-flow-quickstart.html
- SDK on managing runs: https://microsoft.github.io/promptflow/how-to-guides/run-and-evaluate-a-flow/manage-runs.html
- SDK on how to develop evaluation flows: https://microsoft.github.io/promptflow/how-to-guides/develop-a-dag-flow/develop-evaluation-flow.html
- SDK how to on running and evaluating flows: https://microsoft.github.io/promptflow/how-to-guides/run-and-evaluate-a-flow/index.html
- SDK on Getting started with prompty: https://microsoft.github.io/promptflow/tutorials/prompty-quickstart.html

- Microsoft Learn on how evaluation flows: https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/how-to-develop-an-evaluation-flow?view=azureml-api-2#metrics-logging-and-aggregation-node
- Microsoft Learn on custom evaluations: https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/how-to-develop-an-evaluation-flow?view=azureml-api-2
- Example Flex Flow Chat with Functions: https://github.com/microsoft/promptflow/blob/main/examples/flex-flows/chat-with-functions/README.md
- Example Flex Flow EvalFlow class example: https://github.com/microsoft/promptflow/blob/main/examples/flex-flows/eval-checklist/check_list.py
- Open Source GitHub Prompt flow Examples on prompty: https://github.com/microsoft/promptflow/blob/2f6eccf46341d3b4141c0a35bac009130e29e08e/examples/prompty/basic/prompty-quickstart.ipynb
- Open Source GitHub Prompt flow Tutorial on Quality Improvement: https://github.com/microsoft/promptflow/blob/main/examples/tutorials/flow-fine-tuning-evaluation/promptflow-quality-improvement.md
- Open Source GitHub Prompt flow Tutorial on Tracing (very useful for flex flows + AutoGen): https://github.com/microsoft/promptflow/tree/main/examples/tutorials/tracing

### File reference

| Category| Path(s) | Description |
| --- | --- | --- |
| prompty | chat_with_tools.prompty | The generative AI prompt used for the main chat function. |
| prompty | eval.prompty | The generative AI prompt used for the evaluation function. |
| prompty | restate_question.prompty | The generative AI prompt used to restate the question. |
| prompty | tools.json | The set of tools that are passed into the generative AI call through prompty. |
| testing | data.jsonl | Data file used to test the chat function and evaluate the responses. |
| entry | flow.flex.yaml | When prompty is used, yaml is not strictly required; however, for automated deployment, this may be required and is recommended. |
| entry | my_flow_entry.py | The main python function. |
| testing | sample.json | A sample chat history using this flow. |
| other | README.md | Beginning instructions for this lab. |
| other | requirements.txt | Library dependencies. |


Load Env

In [4]:
import os
from dotenv import load_dotenv

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

Set working directory

In [5]:
# Modify the second line as needed to get to the root of your prompt flow directory
%cd "C:\Users\aprilhazel/Source/promptflow_codefirst_llmops/labs/lab03"
%pwd

C:\Users\aprilhazel\Source\promptflow_codefirst_llmops\labs\lab03


'C:\\Users\\aprilhazel\\Source\\promptflow_codefirst_llmops\\labs\\lab03'

Create connection

In [1]:
from promptflow.client import PFClient

# initialize the client
pf = PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

open_ai_connection (AzureOpenAI)


In [2]:
# create needed connection
from promptflow.entities import AzureOpenAIConnection, OpenAIConnection

my_conn_name = "open_ai_connection"
my_api_base="https://coe-openai-sweden.openai.azure.com/"
my_api_version="2024-02-01"

try:
    result = pf.connections.get(name=my_conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=my_conn_name,
        api_key="<user-input>",
        api_base=my_api_base,
        api_type="azure",
        api_version=my_api_version,
    )

    # use this if you have an existing OpenAI account
    # connection = OpenAIConnection(
    #     name=conn_name,
    #     api_key="<user-input>",
    # )
    result = pf.connections.create_or_update(connection)
    print("successfully created connection")

# print the connection details
print(result)


using existing connection
auth_mode: key
name: open_ai_connection
module: promptflow.connections
created_date: '2024-06-18T09:13:26.519864'
last_modified_date: '2024-06-18T15:28:38.626652'
type: azure_open_ai
api_key: '******'
api_base: https://coe-openai-sweden.openai.azure.com/
api_type: azure
api_version: '2024-02-01'



Read prompty file into an object.

Consider the "tools" listed and review the tools.json file.

In [5]:
# Read prompty file
with open("chat_with_tools.prompty") as fin:
    print(fin.read())

---
name: Chat with Tools
description: A basic prompt with tools
model:
    api: chat
    configuration:
      connection: open_ai_connection
      type: azure_openai
      azure_deployment: ${env:CHAT_MODEL_DEPLOYMENT_NAME}
    parameters:
      max_tokens: 128
      temperature: 0.2
      tools: ${file:tools.json}
      tool_choice: auto
inputs:
  question:
    type: string
  chat_history:
    type: list
    default: []
sample:
  question: What's the weather in Beijing?
  chat_history: []
---
# system:
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

{% for item in chat_history %}
{{item.role}}:
{{item.content}}
{% endfor %}

user:
{{question}}



Execute the flow via prompty as a function

In [6]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="chat_with_tools.prompty")

# execute the flow as function
response_message={}
response_message = f(question="What is the weather in France?")
response_message

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_qGyxXkz2vpQwixgskDgHNwo5',
   'function': {'arguments': '{"location":"Paris, France","format":"celsius"}',
    'name': 'get_current_weather'},
   'type': 'function'}]}

Add programatic know-how. For example, the generative AI flow returns the above tool_calls which are then executed dynamically via the function located here: my_flow_entry.py > chat > run_function. The below shows an example of how this is accomplished.

In [16]:
import json
import my_flow_entry

In [20]:
# similar logic to what is used in the my_fow_entry.py file
if "tool_calls" in response_message and len(response_message["tool_calls"]) == 1:
    call = response_message["tool_calls"][0]
    function = call["function"]
    function_name = function["name"]
    function_args = json.loads(function["arguments"])
    print(f"Executing my_flow_entry module > function: {function_name} with Arguments: {function_args}. Result is...")
    function_to_call = getattr(my_flow_entry, function_name)
    result = function_to_call(**function_args) 
    print(result)  

Executing my_flow_entry module > function: get_current_weather with Arguments: {'location': 'Paris, France', 'format': 'celsius'}. Result is...
{'location': 'Paris, France', 'temperature': '72', 'format': 'celsius', 'forecast': ['sunny', 'windy']}


Start a trace session

In [21]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Starting prompt flow service...
You can stop the prompt flow service with the following command:'pf service stop'.
Alternatively, if no requests are made within 1 hours, it will automatically stop.


# Submit a prompt flow run

Run the flow.flex.yaml flow using the default inputs

In [36]:
!pf flow test --flow ./myflow.flex.yaml 

Prompt flow service has started...
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=lab03&uiTraceId=0x34f39e57920750f5a5219b8d2c134c92
2024-06-19 22:55:13 -0500   22048 execution.flow     INFO     [Flex in line 0 (index starts from 0)] stdout> {'location': 'Beijing', 'format': 'celsius'}
{'location': 'Beijing', 'temperature': '72', 'format': 'celsius', 'forecast': ['sunny', 'windy']}


Run the flow.flex.yaml flow using inputs

In [40]:
!pf flow test --flow ./myflow.flex.yaml --inputs question="What is the weather in Paris?"

Prompt flow service has started...
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=lab03&uiTraceId=0xf44cd788f3d35ad14ffe187800588e7d
2024-06-19 23:00:51 -0500   28100 execution.flow     INFO     [Flex in line 0 (index starts from 0)] stdout> {'location': 'Paris', 'format': 'celsius'}
{'location': 'Paris', 'temperature': '72', 'format': 'celsius', 'forecast': ['sunny', 'windy']}


Run the flow as function

In [42]:
from my_flow_entry import chat

In [43]:
# run the flow as function
result = chat(question="What's the weather of Seattle? I love that place!")
# note the type is generator object as we enabled stream in prompty
result

{'location': 'Seattle, WA', 'format': 'fahrenheit'}


"{'location': 'Seattle, WA', 'temperature': '72', 'format': 'fahrenheit', 'forecast': ['sunny', 'windy']}"

# Submit a batch run

Run

In [45]:
from my_flow_entry import chat 
data = "./data.jsonl"  # path to the data file

batch_run = pf.run(
    flow=chat,
    data=data,
    stream=True,
    column_mapping={"question": "${data.question}", "chat_history": "${data.chat_history}"},
)

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=lab03_20240619_230508_028738


[2024-06-19 23:05:12 -0500][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run lab03_20240619_230508_028738, log path: C:\Users\aprilhazel\.promptflow\.runs\lab03_20240619_230508_028738\logs.txt


2024-06-19 23:05:12 -0500   22100 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-19 23:05:12 -0500   22100 execution.bulk     INFO     Current system's available memory is 8332.734375MB, memory consumption of current process is 206.37890625MB, estimated available worker count is 8332.734375/206.37890625 = 40
2024-06-19 23:05:12 -0500   22100 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 40}.
2024-06-19 23:05:17 -0500   22100 execution.bulk     INFO     Process name(SpawnProcess-10)-Process id(30464)-Line number(0) start execution.
2024-06-19 23:05:17 -0500   22100 execution.bulk     INFO     Process name(SpawnProcess-12)-Process id(19316)-Line number(1) start execution.
2024-06-19 23:05:17 -0500   22100 execution.bulk     INFO     Process name(SpawnProcess-11)-Process id

Get details

In [46]:
# get the inputs/outputs details of a finished run.
details = pf.get_details(batch_run)
details.head(10)

,inputs.question,inputs.chat_history,inputs.line_number,inputs.max_total_token,outputs.output
0,How about London next week?,[],0,2048,"{'location': 'London', 'temperature': '60', 'f..."
1,"What will the weather be in St Louis, Missouri...",[],1,2048,"{'location': 'St Louis, MO', 'temperature': '6..."
2,What is the capital of Japan?,[],2,2048,The capital of Japan is Tokyo.


Add batch evaluations

In [52]:
import json

# set eval flow path
eval_flow = "./eval.prompty"
data= "./data.jsonl"

# run the flow with existing run
eval_run = pf.run(
    flow=eval_flow,
    data=data,
    run=batch_run,
    column_mapping={  # map the url field from the data to the url input of the flow
      "chat_history": "${data.chat_history}",
      "question": "${data.question}",
      "answer": "${run.outputs.output}", 
      "ground_truth": "${data.ground_truth}"
      }
)

# stream the run until it's finished
pf.stream(eval_run)

[2024-06-19 23:08:48 -0500][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run lab03_20240619_230848_620512, log path: C:\Users\aprilhazel\.promptflow\.runs\lab03_20240619_230848_620512\logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=lab03_20240619_230848_620512
2024-06-19 23:08:48 -0500   22100 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-06-19 23:08:48 -0500   22100 execution.bulk     INFO     Current system's available memory is 8391.48046875MB, memory consumption of current process is 208.5MB, estimated available worker count is 8391.48046875/208.5 = 40
2024-06-19 23:08:48 -0500   22100 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 40}.
2024-06-19 23:08:53 -0500   22100 execution.bulk     INFO     Process name(SpawnProcess-14)-Process id(18828)-Line number(0) start execution.
2024-06-19 23:08:53 -0500   22100 execution.bulk     INFO     Process name(SpawnProcess-16)-Process id(25176

Get details (Tip: Use tabulate to see the long text fields)

In [55]:
from tabulate import tabulate

# get the inputs/outputs details of a finished run.
eval_details = pf.get_details(eval_run)
# details.head(10)
print(tabulate(eval_details.head(), headers="keys", tablefmt="grid"))

+----+----------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------------------+----------------------+-----------------+----------------------------------------------------------------------------------------------------------------------------------+
|    | inputs.question                                          | inputs.answer                                                                                                   | inputs.ground_truth                                                                                | inputs.chat_history   |   inputs.line_number |   outputs.score | outputs.explanation                                                                                                              |
+====+========================================

Metrics #TODO Not logging locally

In [19]:
# view the metrics of the eval run
metrics = pf.get_metrics(eval_run)
metrics
#print(json.dumps(metrics, indent=4))


{}

# Visualize both the base run and the eval run
(Uses local directories: *C:\Users\<user name>\.promptflow*)

In [56]:
# # visualize both the base run and the eval run
pf.visualize([batch_run, eval_run])

Prompt flow service has started...
The HTML file is generated at 'C:\\Users\\aprilhazel\\AppData\\Local\\Temp\\pf-visualize-detail-smyzdokn.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
